In [1]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q langdetect
# !pip install -q tensorflow
# !pip install -q tf-keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pandas as pd
from pathlib import Path
import shutil
from langdetect import detect
import re
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import numpy as np

###Section 2

In [ ]:
!unzip '/content/drive/MyDrive/archive (2).zip'

Archive:  /content/drive/MyDrive/archive (2).zip
replace tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
data_path = Path('/content')
tweet_data_path = data_path / 'tweets.csv'

tweet_data = pd.read_csv(tweet_data_path,
                         sep=';' ,
                          usecols=['timestamp','text'],
                          on_bad_lines='skip')

<ipython-input-5-56c0db82a36e>:4: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet_data = pd.read_csv(tweet_data_path,


In [ ]:
tweet_data['text'] = tweet_data['text'].astype(str)
tweet_data['timestamp'] = pd.to_datetime(tweet_data['timestamp'])
min(tweet_data['timestamp']),max(tweet_data['timestamp'])

(Timestamp('2007-04-19 07:14:38+0000', tz='UTC'),
 Timestamp('2019-11-23 15:45:57+0000', tz='UTC'))

In [ ]:
price = pd.read_csv('/content/BTC_USD.csv')
price['Date'] = pd.to_datetime(price['Date'])

min_date = min(price['Date']).tz_localize('UTC')
max_date = max(price['Date']).tz_localize('UTC')
print(f'{min_date}\n{max_date}')

tweet_data = tweet_data[(tweet_data['timestamp'] >= min_date) & (tweet_data['timestamp'] <= max_date) ]


2015-01-01 00:00:00+00:00
2025-01-26 00:00:00+00:00


In [ ]:
path = Path('/content/drive/MyDrive')
tweet_data.to_csv(path/'tweets_2015_2019.csv',index=False)

###Section 3

In [ ]:
import gc

In [ ]:
# data_path = Path('/content/drive/MyDrive/Bitcoin_data')
# tweet_data_path = data_path / 'tweets_2015_2019_cleaned.csv'

# tweet_data = pd.read_csv(tweet_data_path)

In [ ]:
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & map symbols
    u"\U0001F700-\U0001F77F"  # Alchemical symbols
    u"\U0001F780-\U0001F7FF"  # Geometric shapes
    u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA00-\U0001FA6F"  # Chess symbols
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251"  # Enclosed characters
    "]+", flags=re.UNICODE
)

emoticon_pattern = re.compile(
    r"(:-\)|:\)|:-\(|:\(|:-D|:D|;-D|;-P|;P|:-P|:P|:-O|:O|:-/|:/|:-\||:\||:-\*|:\*)"
)



tweet_data['text'] = tweet_data['text'].str.replace(emoji_pattern, '', regex=True) # Replace Emojis with an Empty String
tweet_data['text'] = tweet_data['text'].str.replace(emoticon_pattern, '', regex=True)
tweet_data['text'] = tweet_data['text'].str.replace('https?\.?\S+','',regex=True) # Removing the url from the text
tweet_data['text'] = tweet_data['text'].str.replace('#','',regex=True) # removing # from the data
tweet_data['text'] = tweet_data['text'].str.replace('RT',' ',regex=True) # Removing the Re-tweet mark
tweet_data['text'] = tweet_data['text'].str.replace('@[\w]','',regex=True) #removing the twitter handles @user
tweet_data['text'] = tweet_data['text'].str.lower()


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_pattern = r'\b(?:' + '|'.join(map(re.escape, stop_words)) + r')\b'
tweet_data['text'] = (
    tweet_data['text']
    .str.replace(stop_pattern, ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces
    .str.strip()  # Trim leading/trailing spaces
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tweet_data.to_csv('/content/drive/MyDrive/tweets_2015_2019_cleaned.csv', index=False)

In [ ]:
def detect_lan(text):
    try:
        return detect(text)
    except:
        return np.nan
lan = tweet_data['text'].astype(str).apply(detect_lan)

In [ ]:
tweet_data.to_csv('/content/drive/MyDrive/tweets_2015_2019_cleaned.csv', index=False)

np.False_

In [ ]:
tweet_data['lan'] = lan
tweet_data.to_csv('/content/drive/MyDrive/tweets_2015_2019_cleaned.csv', index=False)

###Section 4

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [ ]:
tqdm.pandas()

In [ ]:
text = "Good night 😊"

encoded_input = tokenizer(tweet_data['text'].to_list(),
                          return_token_type_ids=False, # This tokenizer does not take token_type_ids
                          padding=True, # Pad the sequences to the same length
                          truncation=True, # Truncate sequences longer than the model's maximum input length
                          return_tensors='tf' # Return TensorFlow tensors
                          )
output = model(**encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

KeyboardInterrupt: 

In [ ]:
output

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[-1.0439562 ,  0.67082566,  0.23109104],
       [-1.5222687 ,  1.5960919 , -0.31950748],
       [-1.220453  ,  1.3424112 , -0.34673446],
       [-0.5133103 ,  1.0642624 , -0.9422849 ],
       [-0.9380051 ,  1.5546212 , -0.84044486],
       [-0.21034335,  0.09854041, -0.02339912],
       [-0.05127509,  1.1059114 , -1.3113911 ],
       [-0.63617194, -0.3985102 ,  0.9104611 ],
       [-0.04667827, -0.06678388, -0.152857  ],
       [-0.73122495, -0.04770188,  0.59767455]], dtype=float32)>, hidden_states=None, attentions=None)

In [ ]:
def process_tweets_in_batches(tweet_data, tokenizer, model, batch_size=10):
    """Processes tweets in batches for sentiment analysis.

    Args:
        tweet_data (pd.DataFrame): DataFrame containing the tweets.
        tokenizer (transformers.AutoTokenizer): Tokenizer for the model.
        model (transformers.TFAutoModelForSequenceClassification): Sentiment analysis model.
        batch_size (int, optional): Number of tweets to process per batch. Defaults to 10.

    Returns:
        np.ndarray: Sentiment scores for all tweets.
    """
    all_scores = []
    for i in tqdm(range(0, len(tweet_data), batch_size)):
        batch_tweets = tweet_data['text'][i:i + batch_size].to_list()
        encoded_input = tokenizer(
            batch_tweets,
            return_token_type_ids=False,
            padding=True,
            truncation=True,
            return_tensors='tf'
        )
        output = model(**encoded_input)
        scores = output[0].numpy()  # Get scores for the entire batch
        scores = softmax(scores, axis=1)  # Apply softmax along the class dimension
        all_scores.extend(scores)
    return np.array(all_scores)

# Example usage:
batch_size = 512  # Adjust batch size as needed
all_sentiment_scores = process_tweets_in_batches(tweet_data, tokenizer, model, batch_size)
print(all_sentiment_scores)

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.predict(tweet_data['text'].tolist(), batch_size=batch_size)

ValueError: in user code:

    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 2436, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 2421, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 2409, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 2377, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filetkfkkemj.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_genera

In [ ]:
all_scores = softmax(model.predict(tweet_data['text'].tolist(), batch_size=batch_size), axis=1)

In [ ]:
price_data_path = data_path / 'BTC_USD.csv'
price_data = pd.read_csv(price_data_path)